# Tourism Finder
## Applied Data Science Capstone

## Introduction
Tourism finder is an application that is used to find Tourism given any city. Finding tourism hotspots in a city is really important because several businesses can thrive in tourism hotspots. Here is a sample list of businesses that can thrive in tourism hotspots:

1.	Planning a Vacation
2.	Fancy Restaurants 
3.	Nightclubs 
4.	Hotels and Star Hotels 
5.	Malls 
6.	Souvenir Shops 
7.	And more... 

This application is designed to take in the city name as input and an interactive map of the region will be generated as output with the tourism attractions superimposed over the map as circles. 

In [1]:
# Deal with import statements first 
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium 

#!conda install -c anaconda lxml --yes
import lxml

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

# For the Foursquare API
CLIENT_ID = 'censored-sensitive-information'
CLIENT_SECRET = 'censored-sensitive-information'
VERSION = '20180604'
LIMIT = 80
print("Hello Tourism Finder!")

Hello Tourism Finder!


## Dashboard
You will set parameters here. In fact, this dashboard can be replaced with any other kind to get the user input, example, Command-Line Interface, GUI. 

In [2]:
# Set city name 
city = "Singapore"

# Or set latitude and longitude manually 
# and set city to None by uncommenting the line
lat = 13.0827
lon = 80.2707
#city = None

### Step 1: Retrieve coordinates 

In [3]:
if city != None:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(city)
    #print(type(location))
    lat = location.latitude
    lon = location.longitude
    print ("The coordinates of %s are (%f, %f)." % (city, lat, lon))
else:
    print ("Your chosen coordinates are (%f, %f)." % (lat, lon))

The coordinates of Singapore are (1.357107, 103.819499).


### Step 2: Retrieve list of tourism attractions

#### Retrieve by using Foursquare API Call

In [4]:
search_query = 'Tourism'
radius = 50000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lon, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
print("Status code: %d."%results['meta']['code'])

Status code: 200.


#### Process the JSON file

In [5]:
# Assign relevant part of JSON to venues
venues = results['response']['venues']

# Tranform venues into a dataframe
dataframe = json_normalize(venues)
print("%d Results loaded." % dataframe.shape[0])
print("Displaying first 3 rows:")
dataframe.head(3)

50 Results loaded.
Displaying first 3 rows:


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",False,4cff4c65f7b38cfa9a7cc8c3,370 Orchard Road,SG,Singapore,Singapore,NaN,5817,"[370 Orchard Road, 238870, Singapore]","[{'label': 'display', 'lat': 1.305928, 'lng': ...",1.305928,103.830079,238870,NaN,Tourism Authority Of Thailand,v-1592130282
1,"[{'id': '52e81612bcbc57f1066b7a13', 'name': 'N...",False,52e0d6f4498e23eedfbea071,Mandai Lake Road,SG,Singapore,Singapore,NaN,6621,"[Mandai Lake Road, Singapore]","[{'label': 'display', 'lat': 1.405894323317632...",1.405894,103.785460,NaN,NaN,Mandai Eco-Tourism Hub,v-1592130282
2,"[{'id': '4bf58dd8d48988d1a0941735', 'name': 'C...",False,4c7b330176ce9c743c6fb60c,"#06-21/26, Singapore Shopping Center",SG,Singapore,Singapore,190 Clemenceau Ave,7087,"[#06-21/26, Singapore Shopping Center (190 Cle...","[{'label': 'display', 'lat': 1.298463226550203...",1.298463,103.844285,239924,NaN,Tourism Management Institute of Singapore (TMIS),v-1592130282


#### Fix the Categories column

In [6]:
# Fix the Categories Column

# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Apply filter to the categories column
dataframe['categories'] = dataframe.apply(get_category_type, axis=1)
dataframe.head(3)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,Embassy / Consulate,False,4cff4c65f7b38cfa9a7cc8c3,370 Orchard Road,SG,Singapore,Singapore,NaN,5817,"[370 Orchard Road, 238870, Singapore]","[{'label': 'display', 'lat': 1.305928, 'lng': ...",1.305928,103.830079,238870,NaN,Tourism Authority Of Thailand,v-1592130282
1,Nature Preserve,False,52e0d6f4498e23eedfbea071,Mandai Lake Road,SG,Singapore,Singapore,NaN,6621,"[Mandai Lake Road, Singapore]","[{'label': 'display', 'lat': 1.405894323317632...",1.405894,103.785460,NaN,NaN,Mandai Eco-Tourism Hub,v-1592130282
2,College Classroom,False,4c7b330176ce9c743c6fb60c,"#06-21/26, Singapore Shopping Center",SG,Singapore,Singapore,190 Clemenceau Ave,7087,"[#06-21/26, Singapore Shopping Center (190 Cle...","[{'label': 'display', 'lat': 1.298463226550203...",1.298463,103.844285,239924,NaN,Tourism Management Institute of Singapore (TMIS),v-1592130282


### Step 3: Plot them onto a map

In [7]:
# Initialise Map object 
tourism_map = folium.Map(location=[lat, lon], zoom_start=11)

# Red circle represents center location 
if city != None:
    lbl = folium.Popup(city + " (" + str(lat) + ", " + str(lon) + ")", parse_html=True)
else: 
    lbl = folium.Popup(" (" + str(lat) + ", " + str(lon) + ")", parse_html=True)
folium.features.CircleMarker([lat, lon], radius=5, popup=lbl, fill=True, color='red', 
                             fill_color='red', fill_opacity=0.6).add_to(tourism_map)

# Blue circles represent tourism spots
for lati, lngi, cat, attr in zip(dataframe['location.lat'],dataframe['location.lng'],dataframe['categories'], dataframe['name']):
    if cat == None:
        lbl = folium.Popup(str(attr), parse_html=True)
    else:
        lbl = folium.Popup(str(attr) + ' (' + str(cat) + ')', parse_html=True)
    folium.features.CircleMarker([lati, lngi], radius=5, popup=lbl, fill=True, color='blue', 
                                 fill_color='blue', fill_opacity=0.6).add_to(tourism_map)

tourism_map